# Pipeline for Recovering Neural Activity from Calcium Imaging Data

### Relevant Neuroscience Citations / Dependencies
**Motion Correction ->** Suite2P Package from Stringer Lab <br>
**Denoising ->** DeepCAD Package from Dai Lab <br>

**ROI Identification ->** Cellpose Package from Stringer Lab <br>
**Signal Extraction ->** FISSA Package from Rochefort Lab <br>
**Source-Separation ->** FISSA Package from Rochefort Lab <br>
**Spike Inference ->** Cascade Package from Helmchen Lab <br>

## Imports

In [ ]:
from ExperimentManagement.ExperimentHierarchy import ExperimentData
import numpy as np

## Generation of Hierarchy

In [ ]:
# EH = ExperimentData(Mouse="EM0122", Directory="D:\\EM0122")

## Load Hierarchy

In [ ]:
EH = ExperimentData.loadHierarchy("D:\\EM0137")

## Generation of Behavioral Stage

In [1]:
from BehavioralAnalysis.BurrowFearConditioning import FearConditioning # Create your own usable class by inheriting "BehavioralStage" methods alongside your own attributes and functions

In [ ]:
EH.Retrieval = FearConditioning(EH.passMeta(), "Retrieval")

## Image PreProcessing

Imports

In [ ]:
from ImagingAnalysis.PreprocessingImages import PreProcessing

Define Directories

In [ ]:
RawVideoDirectory = EH.Retrieval.folder_dictionary.get("raw_imaging_data")
OutputDirectory = EH.Retrieval.folder_dictionary.get("compiled_imaging_data_folder").path

Import MetaData

In [ ]:
EH.Retrieval.loadBrukerMetaData()

Repackage bruker files into a single tiff

In [ ]:
PreProcessing.repackageBrukerTiffs(RawVideoDirectory, OutputDirectory)
EH.Retrieval.update_folder_dictionary()

Load Images

In [ ]:
images = PreProcessing.loadAllTiffs(OutputDirectory)

Filter Images

In [ ]:
images = PreProcessing.blockwiseFastFilterTiff(images, Footprint=np.ones((7, 3, 3)))

Remove Shutter Artifact

In [ ]:
images = PreProcessing.removeShuttleArtifact(images, chunk_size=7000, artifact_length=1000)

Save Images as Binary

In [ ]:
PreProcessing.saveRawBinary(images, OutputDirectory)
EH.Retrieval.update_folder_dictionary()
EH.recordMod("Repackaged, filtered, and exported images as raw binary. Made video even length this time")

## Image Processing

Construct Folder for Data

In [ ]:
EH.Retrieval.addImageSamplingFolder(30)

Motion Correction

In [ ]:
from ImagingAnalysis.Suite2PAnalysis import Suite2PModule

In [ ]:
MotionCorrection = Suite2PModule(EH.Retrieval.folder_dictionary.get("compiled_imaging_data_folder").path, EH.Retrieval.folder_dictionary.get("imaging_30Hz").path, file_type="binary")
MotionCorrection.motionCorrect()
MotionCorrection.exportCroppedCorrection(MotionCorrection.ops)
del MotionCorrection # Clean Up

Denoise

In [ ]:
from ImagingAnalysis.Denoising import DenoisingModule

In [ ]:
Denoiser = DenoisingModule("ModelForPyTorch", "binary_video",
                    model_path="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\pth",
                    data_path=EH.Retrieval.folder_dictionary.get("imaging_30Hz").folders.get("plane0"),
                    output_path=EH.Retrieval.folder_dictionary.get("imaging_30Hz").folders.get("denoised"),
                    image_type="binary",
                    length="14000",
                    workers=4,
                    vram=24,
                    batch_size2=4)
Denoiser.runDenoising()
Denoiser = None

ROI Detection & Classification

In [ ]:
from ImagingAnalysis.Suite2PAnalysis import Suite2PModule

In [ ]:
S2P = Suite2PModule(EH.Retrieval.folder_dictionary.get("imaging_30Hz").folders.get("denoised"), EH.Retrieval.folder_dictionary.get("imaging_30Hz").path, file_type="binary")
S2P.roiDetection()
S2P.extractTraces()
S2P.classifyROIs()
S2P.spikeExtraction() # Finalize (Required spks.npy to use GUI)
S2P.integrateMotionCorrectionDenoising()

In [ ]:
S2P.iscell, S2P.stat = S2P.remove_small_neurons(S2P.iscell, S2P.stat)
S2P.save_files()
EH.Retrieval.recordMod()
EH.recordMod("S2P Retrieval")
EH.saveHierarchy()
del S2P

Fissa: Signal Extraction

In [ ]:
from ImagingAnalysis.FissaAnalysis import FissaAnalysis

In [ ]:
Fissa = FissaAnalysis(data_folder=EH.Retrieval.folder_dictionary.get("imaging_30Hz").path, video_folder=EH.Retrieval.folder_dictionary.get("imaging_30Hz").folders.get("denoised"))
Fissa.initializeFissa()

In [ ]:
Fissa.extractTraces() # simple, call to extract raw traces from videos
Fissa.saveFissaPrep()

Trace Processing

In [ ]:
from ImagingAnalysis.StaticProcessing import Processing

In [ ]:
# let's smooth the data with edge-preserving to make it nicer
Fissa.ProcessedTraces.smoothed_raw = Processing.smoothTraces_TiffOrg(Fissa.preparation.raw, niter=50, kappa=150, gamma=0.15)[0]
Fissa.preparation.raw = Fissa.ProcessedTraces.smoothed_raw.copy()
#Let's use for separation, so replace the raws with smooths
Fissa.passPrepToFissa()

Fissa: Source-Separation

In [ ]:
Fissa.separateTraces() # simple, call to separate the traces
Fissa.saveFissaSep()

Trace Post-Processing

In [ ]:
# Calculate Fo/F
Fissa.ProcessedTraces.dFoF_result = Processing.calculate_dFoF(Fissa.experiment.result, Fissa.frame_rate, raw=Fissa.preparation.raw, merge_after=False)

# Condense the ROI Traces for each Trial into a Single Matrix
Fissa.ProcessedTraces.merged_dFoF_result = Processing.mergeTraces(Fissa.ProcessedTraces.dFoF_result)

# Detrend the Traces by fitting a 4th-order polynomial and subsequently subtracting
Fissa.ProcessedTraces.detrended_merged_dFoF_result = Processing.detrendTraces(Fissa.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

# Save
Fissa.saveProcessedTraces()
EH.recordMod("Retrieval Source-Separation")
EH.saveHierarchy()

Cascade: Event/Spike/Firing Rate Inference

In [ ]:
from ImagingAnalysis.CascadeAnalysis import CascadeModule

In [ ]:
Cascade = CascadeModule(Fissa.ProcessedTraces.detrended_merged_dFoF_result, Fissa.frame_rate, EH.Retrieval.folder_dictionary.get("imaging_30Hz").folders.get("cascade"), model_folder="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models")

In [ ]:
# Pull Available Models
list_of_models = Cascade.pullModels(Cascade.model_folder)
# Select Model: If you know what model you want, you should use the string instead.
# This model is Global_EXC_10Hz_smoothing_100ms
# Cascade.model_name = list_of_models[21]
Cascade.model_name = "Global_EXC_30Hz_smoothing100ms"
Cascade.downloadModel(Cascade.model_name, "C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models")

In [ ]:
# Infer Spike Probability
Cascade.predictSpikeProb() # Simple, call to infer spike probability for each frame
# Calculate Firing Rates # Simple, firing rate = spike probability * imaging frequency
Cascade.ProcessedInferences.firing_rates = Processing.calculateFiringRate(Cascade.spike_prob, Cascade.frame_rate)

In [ ]:
Cascade.saveSpikeProb(Cascade.save_path)
Cascade.saveProcessedInferences(Cascade.save_path)

In [ ]:
Cascade.inferDiscreteSpikes()

In [ ]:
Cascade.saveSpikeInference(Cascade.save_path)

In [ ]:
EH.recordMod("Retrieval Cascade")
EH.saveHierarchy()

In [ ]:
EH.Retrieval.recordMod()
EH.Retrieval.update_folder_dictionary()
EH.saveHierarchy()
del Fissa
del Cascade